In [6]:
import pandas as pd
from pathlib import Path

# Carpeta donde están los CSV (puede ser "." si es el mismo directorio del script)
DATA_DIR = Path("listos/")
OUTPUT_DIR = Path("composicion/")
# Mapeo estación -> nombre de archivo
archivos_estaciones = {
    "Invierno": "invierno.csv",
    "Verano": "verano.csv",
    "Primavera": "primavera.csv",
    "Otoño": "otoño.csv",   # si tu archivo se llama "otono.csv", cambia aquí
}

# Lista de lagunas (según las columnas de tus archivos)
lagunas = [
    "Termas",
    "Viale",
    "Inta",
    "Gazano",
    "Crespo",
    "Hernandez",
    "Espinillo",
]

def procesar_estacion(nombre_estacion, nombre_archivo, guardar_csv=True):
    ruta = DATA_DIR / nombre_archivo
    if not ruta.exists():
        print(f"\n⚠️ Archivo NO encontrado para {nombre_estacion}: {ruta}")
        return

    print(f"\n==============================")
    print(f"  Estación: {nombre_estacion}")
    print(f"  Archivo:  {ruta}")
    print(f"==============================\n")

    # Leer CSV
    df = pd.read_csv(ruta)

    # Renombrar la primera columna a "Especie"
    primera_col = df.columns[0]
    df = df.rename(columns={primera_col: "Especie"})

    # Limpiar espacios en los nombres de columnas
    df.columns = [c.strip() for c in df.columns]

    # Opcional: eliminar columnas de grupos grandes si no las querés usar
    columnas_excluir = {"Rotiferos", "Copepodos", "Cladoceros"}
    columnas_usables = [c for c in df.columns if c not in columnas_excluir]

    df = df[columnas_usables]

    for laguna in lagunas:
        col_l1 = f"{laguna} L1"
        col_c  = f"{laguna} C"
        col_l2 = f"{laguna} L2"

        # Verificar que existan las columnas (por si alguna laguna no se muestreó en cierta estación)
        if not all(col in df.columns for col in [col_l1, col_c, col_l2]):
            print(f"  ⚠️ Laguna {laguna}: faltan columnas (L1/C/L2). Se omite.\n")
            continue

        # Calcular abundancia total por especie en la laguna (suma de las 3 capas)
        abund_total = (
            df[col_l1].fillna(0) +
            df[col_c].fillna(0) +
            df[col_l2].fillna(0)
        )

        comp = pd.DataFrame({
            "Especie": df["Especie"],
            "Abundancia_total": abund_total
        })

        # Filtrar especies con abundancia > 0
        comp = comp[comp["Abundancia_total"] > 0].copy()

        if comp.empty:
            print(f"  Laguna {laguna}: sin especies registradas (abundancia total = 0).\n")
            continue

        # Calcular porcentaje relativo
        total_laguna = comp["Abundancia_total"].sum()
        comp["Porcentaje"] = comp["Abundancia_total"] / total_laguna * 100

        # Ordenar de mayor a menor
        comp = comp.sort_values("Porcentaje", ascending=False).reset_index(drop=True)

        # Imprimir un resumen en pantalla
        print(f"▶ Laguna: {laguna} — {nombre_estacion}")
        print(f"   Total abundancia (suma de ind/L en L1+C+L2): {total_laguna:.2f}")
        print("   Especies (top 10 por porcentaje):")
        for i, row in comp.head(10).iterrows():
            print(f"    {i+1:2d}. {row['Especie']} — {row['Porcentaje']:.1f}%")
        print("")

        import os

        if guardar_csv:
            os.makedirs(OUTPUT_DIR, exist_ok=True)   # crea carpeta si no existe
            out_name = os.path.join(
                OUTPUT_DIR,
                f"composicion_{laguna}_{nombre_estacion}.csv".replace(" ", "_")
            )
            comp.to_csv(out_name, index=False)

def main():
    for estacion, archivo in archivos_estaciones.items():
        procesar_estacion(estacion, archivo, guardar_csv=True)

if __name__ == "__main__":
    main()



  Estación: Invierno
  Archivo:  listos\invierno.csv

▶ Laguna: Termas — Invierno
   Total abundancia (suma de ind/L en L1+C+L2): 1038.89
   Especies (top 10 por porcentaje):
     1. nauplios  — 25.0%
     2. Copepodo Cyclopoideo sp 1 — 23.5%
     3. Lepadella cf. ovalis — 15.4%
     4. Copepodito  — 14.8%
     5. Keratella tropica — 5.6%
     6. Pompholix triloba  — 5.5%
     7. Lecane bulla  — 5.0%
     8. Moina sp1 — 1.9%
     9. Bdelloideos  — 1.3%
    10. Lepadella patella  — 1.1%

▶ Laguna: Viale — Invierno
   Total abundancia (suma de ind/L en L1+C+L2): 1410.00
   Especies (top 10 por porcentaje):
     1. Lecane luna — 52.2%
     2. Lecane bulla  — 16.5%
     3. Keratella tropica — 15.7%
     4. Mytilina ventralis   — 5.2%
     5. Polyarthra sp. — 4.0%
     6. nauplios  — 2.1%
     7. Diaphanosoma birgei — 1.9%
     8. Copepodito  — 0.7%
     9. Moina sp1 — 0.6%
    10. Trichocerca cf. grandis  — 0.4%

▶ Laguna: Inta — Invierno
   Total abundancia (suma de ind/L en L1+C+L2): 45

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Carpeta donde están los csv individuales
INPUT_DIR = Path("composicion")
# Carpeta de salida para las figuras
PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(exist_ok=True)

ESTACIONES = ["Invierno", "Verano", "Primavera", "Otoño"]
LAGUNAS = ["Termas", "Viale", "Inta", "Gazano", "Crespo", "Hernandez", "Espinillo"]

TOP_N = 20   # nº máx. de especies en el resumen

def plot_horizontal_bars(ax, especies, valores, titulo, xmax=None):
    """Barras horizontales con el nombre al final de la barra."""
    y = np.arange(len(especies))
    ax.barh(y, valores)

    ax.set_yticks([])

    if xmax is None:
        xmax = max(valores) * 1.1 if len(valores) > 0 else 1.0
    # un poco más de margen a la derecha para que entren los nombres
    ax.set_xlim(0, xmax * 1.05)

    for i, (val, esp) in enumerate(zip(valores, especies)):
        ax.text(val + xmax * 0.015, i, str(esp), va='center', fontsize=8)

    ax.set_title(titulo, fontsize=10)
    ax.invert_yaxis()
    ax.tick_params(axis='x', labelsize=8)


def procesar_laguna(laguna: str):
    print(f"\n=== Laguna: {laguna} ===")

    dfs = []
    for est in ESTACIONES:
        file_path = INPUT_DIR / f"composicion_{laguna}_{est}.csv"
        if not file_path.exists():
            print(f"  ⚠ No se encuentra {file_path}, se omite {est}.")
            continue

        df = pd.read_csv(file_path)
        if "Especie" not in df.columns or "Abundancia_total" not in df.columns:
            print(f"  ⚠ Formato inesperado en {file_path}, se omite.")
            continue

        df["Estacion"] = est
        dfs.append(df)

    if not dfs:
        print(f"  ❌ No hay datos válidos para {laguna}.")
        return

    data = pd.concat(dfs, ignore_index=True)

    # -------- Resumen total (todas las estaciones) --------
    resumen_total = (
        data.dropna(subset=["Especie"])
            .groupby("Especie")["Abundancia_total"]
            .sum()
            .reset_index()
    )

    resumen_total = resumen_total[resumen_total["Abundancia_total"] > 0]

    resumen_total["Porcentaje_total"] = (
        resumen_total["Abundancia_total"] /
        resumen_total["Abundancia_total"].sum() * 100
    )

    resumen_total = resumen_total.sort_values(
        "Porcentaje_total", ascending=False
    ).head(TOP_N)

    especies_resumen = resumen_total["Especie"].tolist()
    xmax_global = resumen_total["Porcentaje_total"].max() * 1.3

    # -------- Figura --------
    n_rows = 1 + len(ESTACIONES)
    fig, axes = plt.subplots(
        n_rows, 1,
        figsize=(13, 2 + 0.4 * len(especies_resumen) * n_rows),
        sharex=True
    )
    if n_rows == 1:
        axes = [axes]

    # ticks X en todos
    for ax in axes:
        ax.tick_params(axis='x', which='both', labelbottom=True)

    # Resumen total
    plot_horizontal_bars(
        axes[0],
        especies_resumen,
        resumen_total["Porcentaje_total"].values,
        titulo=f"{laguna} - Todas las estaciones (resumen)",
        xmax=xmax_global
    )

    # -------- Subplots por estación --------
    for i, est in enumerate(ESTACIONES, start=1):
        ax = axes[i]
        df_est = data[data["Estacion"] == est].copy()

        if df_est.empty:
            ax.set_title(f"{laguna} - {est} (sin datos)")
            ax.set_yticks([])
            continue

        est_resume = (
            df_est.dropna(subset=["Especie"])
                  .groupby("Especie")["Abundancia_total"]
                  .sum()
                  .reset_index()
        )

        # solo especies presentes
        est_resume = est_resume[est_resume["Abundancia_total"] > 0]

        if est_resume.empty:
            ax.set_title(f"{laguna} - {est} (sin especies)")
            ax.set_yticks([])
            continue

        total_est = est_resume["Abundancia_total"].sum()
        est_resume["Porcentaje"] = est_resume["Abundancia_total"] / total_est * 100

        # ordenadas por porcentaje de esa estación
        est_resume = est_resume.sort_values("Porcentaje", ascending=False)

        especies_est = est_resume["Especie"].tolist()
        valores_est = est_resume["Porcentaje"].values

        plot_horizontal_bars(
            ax,
            especies_est,
            valores_est,
            titulo=f"{laguna} - {est}",
            xmax=xmax_global
        )

    fig.suptitle(f"Composición porcentual de especies - Laguna {laguna}",
                 fontsize=14)
    plt.tight_layout()
    # un poquito de espacio a la derecha para etiquetas largas
    plt.subplots_adjust(top=0.93, right=0.88)

    out_path = PLOTS_DIR / f"composicion_especies_{laguna}.png"
    fig.savefig(out_path, dpi=300)
    plt.close(fig)
    print(f"  ✔ Figura guardada en: {out_path}")


def main():
    for laguna in LAGUNAS:
        procesar_laguna(laguna)


if __name__ == "__main__":
    main()



=== Laguna: Termas ===


C:\Users\Facu\AppData\Local\Temp\ipykernel_7148\3766634913.py:150: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


  ✔ Figura guardada en: plots\composicion_especies_Termas.png

=== Laguna: Viale ===


C:\Users\Facu\AppData\Local\Temp\ipykernel_7148\3766634913.py:150: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


  ✔ Figura guardada en: plots\composicion_especies_Viale.png

=== Laguna: Inta ===


C:\Users\Facu\AppData\Local\Temp\ipykernel_7148\3766634913.py:150: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


  ✔ Figura guardada en: plots\composicion_especies_Inta.png

=== Laguna: Gazano ===
  ✔ Figura guardada en: plots\composicion_especies_Gazano.png

=== Laguna: Crespo ===


C:\Users\Facu\AppData\Local\Temp\ipykernel_7148\3766634913.py:150: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


  ✔ Figura guardada en: plots\composicion_especies_Crespo.png

=== Laguna: Hernandez ===
  ✔ Figura guardada en: plots\composicion_especies_Hernandez.png

=== Laguna: Espinillo ===
  ✔ Figura guardada en: plots\composicion_especies_Espinillo.png


In [12]:
import pandas as pd
from pathlib import Path
import glob

# Directorio donde se guardaron los CSV generados anteriormente
DATA_DIR = Path("composicion")

# Estaciones esperadas
estaciones = ["Invierno", "Verano", "Primavera", "Otoño"]

# Listado de lagunas
lagunas = ["Termas", "Viale", "Inta", "Gazano", "Crespo", "Hernandez", "Espinillo"]

def concatenar_por_laguna(laguna):
    print(f"\nProcesando laguna: {laguna}")

    dfs = []  # almacenará los df de cada estación
    
    for est in estaciones:
        # Buscar archivos del tipo:
        # composicion_Termas_Invierno.csv
        patron = f"composicion_{laguna}_{estaciones}.csv"
        archivos = list(DATA_DIR.glob(f"composicion_{laguna}_{est}.csv"))

        if not archivos:
            print(f"  ⚠️ No existe archivo para {laguna} en {est}. Se omite estación.")
            continue

        archivo = archivos[0]  # debería haber solo uno
        
        df = pd.read_csv(archivo)

        # Renombrar columna Porcentaje -> estacion_% (ej: Invierno_%)
        df = df[["Especie", "Porcentaje"]].rename(columns={"Porcentaje": f"{est}_%"})

        dfs.append(df)

    if not dfs:
        print(f"  ❌ No hay datos para {laguna}. Se omite.")
        return

    # Merge incremental por "Especie"
    df_final = dfs[0]
    for df_est in dfs[1:]:
        df_final = pd.merge(df_final, df_est, on="Especie", how="outer")

    # Rellenar NaN con 0
    df_final = df_final.fillna(0)

    # Ordenar especies por promedio de porcentajes
    cols_pct = [c for c in df_final.columns if c.endswith("%")]
    df_final["Promedio"] = df_final[cols_pct].mean(axis=1)
    df_final = df_final.sort_values("Promedio", ascending=False).drop(columns=["Promedio"])

    # Guardar archivo final
    out_name = f"composicion_total_{laguna}.csv"
    df_final.to_csv(out_name, index=False)

    print(f"  ✔ Archivo generado: {out_name}")

def main():
    for laguna in lagunas:
        concatenar_por_laguna(laguna)

if __name__ == "__main__":
    main()



Procesando laguna: Termas
  ✔ Archivo generado: composicion_total_Termas.csv

Procesando laguna: Viale
  ✔ Archivo generado: composicion_total_Viale.csv

Procesando laguna: Inta
  ✔ Archivo generado: composicion_total_Inta.csv

Procesando laguna: Gazano
  ✔ Archivo generado: composicion_total_Gazano.csv

Procesando laguna: Crespo
  ✔ Archivo generado: composicion_total_Crespo.csv

Procesando laguna: Hernandez
  ✔ Archivo generado: composicion_total_Hernandez.csv

Procesando laguna: Espinillo
  ✔ Archivo generado: composicion_total_Espinillo.csv


In [13]:
import pandas as pd
from pathlib import Path
import os

INPUT_DIR = Path("composicion")      # tus csv individuales
OUTPUT_DIR = Path("dominancia")        # carpeta nueva para resultados
OUTPUT_DIR.mkdir(exist_ok=True)

LAGUNAS = ["Termas", "Viale", "Inta", "Gazano", "Crespo", "Hernandez", "Espinillo"]
ESTACIONES = ["Invierno", "Verano", "Primavera", "Otoño"]

def generar_dominancia_laguna(laguna):
    print(f"\n⏳ Procesando laguna {laguna}...")

    dfs = []

    for est in ESTACIONES:
        file_path = INPUT_DIR / f"composicion_{laguna}_{est}.csv"
        if not file_path.exists():
            print(f"  ⚠ No existe {file_path}, se omite.")
            continue

        df = pd.read_csv(file_path)
        if "Especie" not in df.columns or "Abundancia_total" not in df.columns:
            print(f"  ⚠ Formato incorrecto en {file_path}, se omite.")
            continue

        df["Estacion"] = est
        dfs.append(df)

    if not dfs:
        print(f"  ❌ No hay datos válidos para {laguna}")
        return

    # Concatenar estaciones
    data = pd.concat(dfs, ignore_index=True)

    # Suma total por especie (todas las estaciones)
    resumen = (
        data.groupby("Especie")["Abundancia_total"]
        .sum()
        .reset_index()
    )

    # Filtrar especies presentes
    resumen = resumen[resumen["Abundancia_total"] > 0]

    # Calcular porcentaje total
    total_abundancia = resumen["Abundancia_total"].sum()
    resumen["Porcentaje_dominancia"] = (resumen["Abundancia_total"] / total_abundancia) * 100

    # Ordenar
    resumen = resumen.sort_values("Porcentaje_dominancia", ascending=False)

    # Guardar
    out_file = OUTPUT_DIR / f"dominancia_total_{laguna}.csv"
    resumen.to_csv(out_file, index=False)
    print(f"  ✔ Archivo guardado: {out_file}")


def main():
    for laguna in LAGUNAS:
        generar_dominancia_laguna(laguna)


if __name__ == "__main__":
    main()



⏳ Procesando laguna Termas...
  ✔ Archivo guardado: dominancia\dominancia_total_Termas.csv

⏳ Procesando laguna Viale...
  ✔ Archivo guardado: dominancia\dominancia_total_Viale.csv

⏳ Procesando laguna Inta...
  ✔ Archivo guardado: dominancia\dominancia_total_Inta.csv

⏳ Procesando laguna Gazano...
  ✔ Archivo guardado: dominancia\dominancia_total_Gazano.csv

⏳ Procesando laguna Crespo...
  ✔ Archivo guardado: dominancia\dominancia_total_Crespo.csv

⏳ Procesando laguna Hernandez...
  ✔ Archivo guardado: dominancia\dominancia_total_Hernandez.csv

⏳ Procesando laguna Espinillo...
  ✔ Archivo guardado: dominancia\dominancia_total_Espinillo.csv
